In [2]:
import cv2
from cv2 import cuda
from math import*
import numpy as np
from numba import jit,njit
import time
import pandas as pd
import matlab.engine
# Khởi động matlab engine
eng = matlab.engine.start_matlab()
print('MALAB ENGINE FINISHED BEGINNING !!!')

MALAB ENGINE FINISHED BEGINNING !!!


In [6]:
import cv2
from cv2 import cuda
# Kiểm tra phiên bản OpenCV
opencv_version = cv2.__version__
print("Phiên bản OpenCV:", opencv_version)
print("Phiên bản OpenCV:", cuda.getCudaEnabledDeviceCount())

Phiên bản OpenCV: 4.9.0
Phiên bản OpenCV: 1


In [2]:

def calculate_trajectory(map,startPoint,goalPoint,space):
    startPoint = matlab.double(startPoint) # [meters, meters, radians]
    goalPoint  = matlab.double(goalPoint)
    map  = matlab.double(map.tolist())
    space = eng.double(space)
    result = eng.Hybrid_Astar(map,startPoint,goalPoint,space)
    result = np.array(result)
    point = result[:,:3]
    point[:,2] = (point[:,2]*180/pi)-90
    direction = result[:,3]
    return point, direction

In [4]:
actual_angle = 0

In [3]:
# Hàm tạo tìm điểm mục tiêu từ các điểm của global state cách vị trí xe hiện tại 90 decimeter
@jit(nopython=True)
def find_goalPose (glo_state,current_x, current_y, current_step ):
    state = np.copy(glo_state)
    x = current_x*2
    y = current_y*2
    for step in range(current_step,len(state)):
        dis = sqrt((state[step,0]-x)**2 + (state[step,1]-y)**2)
        if dis >= 18:
            break
    goal = state[step]
    goal[0]= max(min(20+(goal[0]-x), 39), 0)
    goal[1]= max(min(20+(goal[1]-y), 39), 0)
    goal[2]= (actual_angle+90)*pi/180
    return goal,step

In [4]:
@jit(nopython = True)
def check_collision_goal (loc_map, goal_x, goal_y):
    # initial variable
    safe_space = 2 # khoảng bán kính an toàn khi có vật cản trục x
    map = np.copy(loc_map)
    nearest_r =0
    nearest_l = 0


    obstacle = ''

    loc_x_check = int(goal_x)
    loc_y_check = int(40-goal_y)
    loc_y_up = max(min(loc_y_check-7, 40), 0)
    loc_y_down = max(min(loc_y_check+7, 40), 0)

    
    chk_right = 0# biến đếm cho check collision right
    chk_left = 0# biến đếm cho check collision left
    max_cnt_right = 40 - loc_x_check
    max_cnt_left  = loc_x_check 
    collision = False
    # dò sang bên phải 
    for cnt_right in range(0,max_cnt_right):
        column = map[loc_y_up:loc_y_down,loc_x_check+cnt_right]
        if np.all(column == 0):
            chk_right = chk_right + 1 
            if chk_right == safe_space:
                break
        else: 
            obstacle = 'r'
            chk_right = 0
            if nearest_r ==0:
                nearest_r = cnt_right
            
    for cnt_left in range(0,max_cnt_left):
        column = map[loc_y_up:loc_y_down,loc_x_check-cnt_left]
        if np.all( column == 0):
            chk_left = chk_left + 1 
            if chk_left == safe_space:
                break
        else: 
            obstacle = 'l'
            chk_left = 0
            if nearest_l ==0:
                nearest_l = cnt_left
    if cnt_right > max_cnt_right-2  and  cnt_left>max_cnt_left-2 :
        collision = True
        x_out = 0
        y_out = 0
    
    elif cnt_right > max_cnt_right-2 :
        x_out = goal_x - cnt_left 
        y_out = goal_y
        if obstacle != 'l' :
            x_out = x_out+chk_left
            if nearest_r < safe_space:
                x_out = x_out-(safe_space-nearest_r)
        


    elif cnt_left > max_cnt_left-2:
        x_out = goal_x + cnt_right 
        y_out = goal_y
        if obstacle != 'r' :
            x_out = x_out-chk_right
            if nearest_l < safe_space:
                x_out = x_out+(safe_space-nearest_l)
        

    else:
        if cnt_right <= cnt_left:
            x_out = goal_x + cnt_right
            y_out = goal_y
            if obstacle != 'r':
                x_out = x_out-chk_right
                if nearest_l < safe_space:
                    x_out = x_out+(safe_space-nearest_l)
            
        else:
            x_out = goal_x - cnt_left 
            y_out = goal_y
            if obstacle != 'l' :
                x_out = x_out+chk_left
                if nearest_r < safe_space:
                    x_out = x_out-(safe_space-nearest_r)
    return map, collision, x_out, y_out

In [8]:
glo_map = np.array(pd.read_csv("map_test.csv"))
goalPose = [4, 90, 1.5707963267948966]
s_state = [25.0, 4.0, (0+90)*pi/180]
list_state, list_direction = calculate_trajectory(map = glo_map ,startPoint = s_state, goalPoint = goalPose,space=1)

In [34]:
list_glo_state = np.array(pd.read_csv("list_test.csv"))
goal_point, index = find_goalPose (list_glo_state,current_x = 11.5, current_y= 3, current_step=0 )
goal_point,index

(array([20.        , 38.        ,  1.57079633]), 17)

In [5]:
local_map =np.array(pd.read_csv("map_test.csv"))

local_map[0,15:18]=1

goalPose = [20.0, 38.0, 1.5707963267948966]
s_state = [20, 20, (0+90)*pi/180]




In [31]:
chk_local_map, check_colision, checked_goal_x, checked_goal_y = check_collision_goal (loc_map = local_map, goal_x = goalPose[0], goal_y = goalPose[1])
goalChecked = [checked_goal_x,checked_goal_y, goalPose[2]]
goalChecked,check_colision

([21.0, 29.0, 1.5707963267948966], False)

In [77]:
start = time.time()

chk_local_map, check_colision, checked_goal_x, checked_goal_y = check_collision_goal (loc_map = local_map, goal_x = goalPose[0], goal_y = goalPose[1])
goalChecked = [checked_goal_x,checked_goal_y, goalPose[2]]
print(goalChecked)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

list_state, list_direction = calculate_trajectory(map = chk_local_map ,startPoint = s_state, goalPoint = goalChecked,space=10)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

[21.0, 38.0, 1.5707963267948966]
Elapsed (with compilation) = 0.0
Elapsed (with compilation) = 0.04002261161804199


In [1]:
list_state

NameError: name 'list_state' is not defined

In [ ]:
start1 = time.time()
map = chk_local_map 
startPoint = s_state 
goalPoint = goalChecked
space=5
startPoint = matlab.double(startPoint) # [meters, meters, radians]
end1 = time.time()
print("1 = %s" % (end1 - start1))
goalPoint  = matlab.double(goalPoint)
end1 = time.time()
print("2 = %s" % (end1 - start1))
map  = matlab.double(map.tolist())
end1 = time.time()
print("3 = %s" % (end1 - start1))
space = eng.double(space)
end1 = time.time()
print("4 = %s" % (end1 - start1))
result = eng.Hybrid_Astar(map,startPoint,goalPoint,space)
end1 = time.time()
print("5 = %s" % (end1 - start1))
result = np.array(result)
point = result[:,:3]
point[:,2] = (point[:,2]*180/pi)-90
direction = result[:,3]
end1 = time.time()
print("6 = %s" % (end1 - start1))

1 = 0.0005831718444824219
2 = 0.0005831718444824219
3 = 0.04107260704040527
4 = 0.04707527160644531
5 = 0.3363003730773926
6 = 0.33730101585388184


In [ ]:
list_state

array([[1.00000000e+02, 1.00000000e+02, 1.42108547e-14],
       [9.97487997e+01, 1.04992837e+02, 3.53414646e+00],
       [9.94262544e+01, 1.10215327e+02, 3.53414646e+00],
       [9.91037090e+01, 1.15437817e+02, 3.53414646e+00],
       [9.87811637e+01, 1.20660307e+02, 3.53414646e+00],
       [9.84586183e+01, 1.25882796e+02, 3.53414646e+00],
       [9.81360730e+01, 1.31105286e+02, 3.53414646e+00],
       [9.78135276e+01, 1.36327776e+02, 3.53414646e+00],
       [9.74909823e+01, 1.41550266e+02, 3.53414646e+00],
       [9.71684369e+01, 1.46772756e+02, 3.53414646e+00],
       [9.68458915e+01, 1.51995246e+02, 3.53414646e+00],
       [9.65233462e+01, 1.57217735e+02, 3.53414646e+00],
       [9.62008008e+01, 1.62440225e+02, 3.53414646e+00],
       [9.58782555e+01, 1.67662715e+02, 3.53414646e+00],
       [9.55557101e+01, 1.72885205e+02, 3.53414646e+00],
       [9.52331648e+01, 1.78107695e+02, 3.53414646e+00],
       [9.49106194e+01, 1.83330185e+02, 3.53414646e+00],
       [9.45880740e+01, 1.88552

In [ ]:
global actual_angle 
actual_angle = 0

In [ ]:
def find_goalPose (glo_state,current_x, current_y, current_step ):
    state = np.copy(glo_state)
    current_x = current_x*10
    current_y = current_y*10
    for step in range(current_step,len(state)):
        dis = sqrt((state[step,0]-current_x)**2 + (state[step,1]-current_y)**2)
        if dis >= 90:
            break
    goal = state[step]
    print(goal)
    goal[0]= max(min(100+(goal[0]-current_x), 199), 0)
    goal[1]= max(min(100+(goal[1]-current_y), 199), 0)
    goal[2]= (actual_angle+90)*pi/180
    return goal,step

In [ ]:
list_state_global =np.array([[1.300000000000000000e+02,4.000000000000000000e+01,0.000000000000000000e+00],
[1.270397872751081820e+02,5.460318783657439212e+01,2.291831180523293199e+01],
[1.142743843569894011e+02,8.139253688189347713e+01,7.639437268410972592e+00],
[1.136083346634394786e+02,9.637032946873921446e+01,3.819718634205486296e+00],
[1.118630292927098822e+02,1.112621551342425192e+02,9.549296585513715740e+00],
[1.101177239219802857e+02,1.261539807997458382e+02,3.819718634205486296e+00],
[1.097845140271486457e+02,1.411465750382547810e+02,0.000000000000000000e+00],
[1.097845140271486457e+02,1.561465750382547810e+02,0.000000000000000000e+00],
[1.097845140271486457e+02,1.711465750382547810e+02,0.000000000000000000e+00],
[1.097845140271486457e+02,1.861465750382547810e+02,0.000000000000000000e+00],
[1.097845140271486457e+02,2.011465750382547810e+02,0.000000000000000000e+00],
[1.097845140271486457e+02,2.161465750382547810e+02,0.000000000000000000e+00],
[1.097845140271486457e+02,2.311465750382547810e+02,0.000000000000000000e+00],
[1.087878429192107319e+02,2.460800415780078367e+02,1.145915590261647310e+01],
[1.036636421216672801e+02,2.601178519684649473e+02,2.864788975654117564e+01],
[9.821265550402722511e+01,2.740782447937760935e+02,2.003524715430464198e+01],
[9.303380696070581735e+01,2.882798249331102056e+02,2.003524715430464198e+01],
[8.785495841738440959e+01,3.024814050724442609e+02,2.003524715430464198e+01],
[8.267610987406300183e+01,3.166829852117784867e+02,2.003524715430464198e+01],
[7.749726133074157985e+01,3.308845653511126557e+02,2.003524715430464198e+01],
[7.231841278742017209e+01,3.450861454904467109e+02,2.003524715430464198e+01],
[6.713956424409875012e+01,3.592877256297808799e+02,2.003524715430464198e+01],
[6.196071570077732815e+01,3.734893057691150489e+02,2.003524715430464198e+01],
[5.678186715745592750e+01,3.876908859084491041e+02,2.003524715430464198e+01],
[5.160301861413449132e+01,4.018924660477832731e+02,2.003524715430464198e+01],
[4.642417007081309066e+01,4.160940461871173852e+02,2.003524715430464198e+01],
[4.124532152749166158e+01,4.302956263264516110e+02,2.003524715430464198e+01],
[3.315253058244879014e+01,4.428780390506249773e+02,4.712981514410549266e+01],
[2.000000000000000000e+01,4.500000000000000000e+02,7.600000000000002842e+01]])
list_state_global 

array([[130.        ,  40.        ,   0.        ],
       [127.03978728,  54.60318784,  22.91831181],
       [114.27438436,  81.39253688,   7.63943727],
       [113.60833466,  96.37032947,   3.81971863],
       [111.86302929, 111.26215513,   9.54929659],
       [110.11772392, 126.1539808 ,   3.81971863],
       [109.78451403, 141.14657504,   0.        ],
       [109.78451403, 156.14657504,   0.        ],
       [109.78451403, 171.14657504,   0.        ],
       [109.78451403, 186.14657504,   0.        ],
       [109.78451403, 201.14657504,   0.        ],
       [109.78451403, 216.14657504,   0.        ],
       [109.78451403, 231.14657504,   0.        ],
       [108.78784292, 246.08004158,  11.4591559 ],
       [103.66364212, 260.11785197,  28.64788976],
       [ 98.2126555 , 274.07824479,  20.03524715],
       [ 93.03380696, 288.27982493,  20.03524715],
       [ 87.85495842, 302.48140507,  20.03524715],
       [ 82.67610987, 316.68298521,  20.03524715],
       [ 77.49726133, 330.88456

In [ ]:

p_index = 0
goalPose,index = find_goalPose (glo_state= list_state_global ,current_x = 13, current_y = 4, current_step = p_index)
goalPose,index

[109.78451403 141.14657504   0.        ]


(array([ 79.78451403, 199.        ,   1.57079633]), 6)

In [ ]:
def transformation_matrix(center,arr_point_in):
    point_in = np.copy(arr_point_in)
    point_in = point_in.astype(np.int64)
    array_out = np.array([])
    rotation_matrix = np.array([[1, 0 ],
                                [0, -1],])
    translation_vector = np.array([center[0]*10-100, 750-(center[1]*10-100)])
    for i in range(0, len(point_in)):
        point = np.dot(rotation_matrix, point_in[i]) + translation_vector
        if array_out.size == 0:  # Kiểm tra xem array_out có rỗng không
            array_out = point
        else:
            array_out = np.vstack([array_out, point]) 
    return array_out

In [ ]:
list_state = np.array([[1.000000000000000000e+02,1.000000000000000000e+02,1.421085471520200372e-14],
[9.899650494319288896e+01,1.149647166300241565e+02,4.134635781509260255e+00],
[9.778160178814469816e+01,1.317710221176497214e+02,4.134635781509260255e+00],
[9.656669863309650736e+01,1.485773276052752863e+02,4.134635781509260255e+00],
[9.535179547804830236e+01,1.653836330929008511e+02,4.134635781509260255e+00],
[9.413689232300009735e+01,1.821899385805264444e+02,4.134635781509260255e+00],
[9.300000000000000000e+01,1.990000000000000000e+02,1.421085471520200372e-14]])


In [ ]:
c =np.array([13,4])
list_point = np.int32(transformation_matrix(c,list_state[:,:2]))
list_point

array([[130, 710],
       [128, 696],
       [127, 679],
       [126, 662],
       [125, 645],
       [124, 628],
       [123, 611]])

In [ ]:
import numpy as np
import json

# Tạo mảng numpy với kích thước 200x200
array_2d = np.zeros((200, 200), dtype=np.float64)

# Tính toán dung lượng của mảng NumPy
array_size_bytes = array_2d.nbytes

# Encode mảng thành chuỗi JSON
json_encoded = json.dumps(array_2d.tolist()).encode('utf-8')

# Tính toán dung lượng của chuỗi JSON
json_size_bytes = len(json_encoded)

# In ra kết quả
print("Dung lượng của mảng NumPy:", array_size_bytes, "bytes")
print("Dung lượng của chuỗi JSON encoded:", json_size_bytes, "bytes")

Dung lượng của mảng NumPy: 320000 bytes
Dung lượng của chuỗi JSON encoded: 200400 bytes


In [ ]:
eng.quit()
print("EXIT MATLAB ENGINE !!!")

EXIT MATLAB ENGINE !!!
